# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

from glob import glob
from tqdm.auto import tqdm

# Get Original Data Path

In [2]:
# 원본 데이터가 저장되어진 최상위 폴더의 경로
org_root = "G:/유바이오시스/"

org_id_list = [] # 대상자의 id에 해당하는 정보를 담아두는 리스트 객체
for (root, dirs, files) in tqdm(os.walk(org_root)): # 최상위 폴더의 트리구조를 탐색하는 반복문
    if len(files) > 0: # 하위 폴더에 파일이 1개 이상인 경우만 탐색
        for file_name in files: # 하위 폴더의 파일들을 1개씩 가져오는 반목문
            file_nm = ".".join(file_name.split(".")[:-1]) # 확장자를 제외한 파일명 변수
            if len(file_nm) > 0:
                if file_nm[0] != ".": # 임시파일 여부 확인
                    file_ext = file_name.split(".")[-1] # 파일 확장자 변수
                    if file_ext == "bmp" or file_ext == "txt" or file_ext == "dat":
                        # 원본 id로는 원본 파일의 전체 경로
                        file_path = os.path.join(root, file_name).replace("\\", "/")
                        org_id = "/".join((root,file_nm)).replace("\\","/")

                        if org_id not in org_id_list:
                            org_id_list.append(org_id)
                        else:
                            pass
                    else:
                        pass
                else:
                    pass

0it [00:00, ?it/s]

In [3]:
# id 에 관련한 데이터 프레임 생성 후 원본 id 정보 삽입
data_df = pd.DataFrame(columns=["new_id", "org_id"])
data_df["org_id"] = org_id_list
data_df.tail(3)

,new_id,org_id
85106,NaN,G:/유바이오시스/강남SV/20230413/k1_2320370399_000538440
85107,NaN,G:/유바이오시스/강남SV/20230413/k1_2320370399_000538751
85108,NaN,G:/유바이오시스/강남SV/20230413/k1_2390059976_000538752


In [4]:
data_df.head(3)

,new_id,org_id
0,NaN,G:/유바이오시스/경희의료원/1번장비/1_03312023_000543011
1,NaN,G:/유바이오시스/경희의료원/1번장비/1_10022701009474000540637...
2,NaN,G:/유바이오시스/경희의료원/1번장비/1_I27TS21H0_000540899


# 데이터 비식별화

In [5]:
# 비식별화 id는 u로 시작하고 숫자 7개를 부여하는데, 데이터프레임의 index 번호를 사용하여 id 생성
new_id_list = []
for idx in tqdm(range(len(data_df))):
    new_id = "".join(("u", str(idx).zfill(7)))
    new_id_list.append(new_id)
data_df["new_id"] = new_id_list
data_df.tail(3)

  0%|          | 0/85109 [00:00<?, ?it/s]

,new_id,org_id
85106,u0085106,G:/유바이오시스/강남SV/20230413/k1_2320370399_000538440
85107,u0085107,G:/유바이오시스/강남SV/20230413/k1_2320370399_000538751
85108,u0085108,G:/유바이오시스/강남SV/20230413/k1_2390059976_000538752


In [6]:
data_df.to_csv("G:/Ubiosis/Ubiosis_data_list.csv",index=False)

# Data Reconstruction

In [7]:
# 비식별화 id를 기준으로 데이터 폴더 구출
## /recon/비식별화id/dat, bmp, txt

# 데이터 구축할 폴더 생성
data_path = "G:/Ubiosis/recon/"
if os.path.isdir(data_path) == False:
    os.makedirs(data_path)
    
# 비식별화 id 데이터프레임을 한 index 씩 탐색하는 반복문
for idx in tqdm(range(len(data_df))):
    org_id = data_df["org_id"][idx] # 원본 파일 경로
    dat_file = ".".join((org_id, "dat")) # 원본 dat 파일 경로
    bmp_file = ".".join((org_id, "bmp")) # 원본 bmp 파일 경로
    txt_file = ".".join((org_id, "txt")) # 원본 txt 파일 경로

    new_id = data_df["new_id"][idx] # 비식별화 id
    new_id_path = os.path.join(data_path, new_id) # 비식별화 데이터 경로
    if os.path.isdir(new_id_path) == False:
        os.mkdir(new_id_path)
    # dat 파일을 비식별화 id를 이용한 파일명으로 복사/붙여넣기
    if os.path.isfile(dat_file) == True:
        new_file_nm = ".".join((new_id, "dat"))
        new_file = os.path.join(new_id_path, new_file_nm)
        shutil.copy(dat_file, new_file)
    # bmp 파일을 비식별화 id를 이용한 파일명으로 복사/붙여넣기
    if os.path.isfile(bmp_file) == True:
        new_file_nm = ".".join((new_id, "bmp"))
        new_file = os.path.join(new_id_path, new_file_nm)
        shutil.copy(bmp_file, new_file)
    # bmp 파일을 비식별화 id를 이용한 파일명으로 복사/붙여넣기
    if os.path.isfile(txt_file) == True:
        new_file_nm = ".".join((new_id, "txt"))
        new_file = os.path.join(new_id_path, new_file_nm)
        shutil.copy(txt_file, new_file)

  0%|          | 0/85109 [00:00<?, ?it/s]